<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Building an Application with SingleStoreDB</h1>
    </div>
</div>

# Pre-AI Email Marketing - "Spray and Pray"

We've all received these forms of emails where it's clear that they just send out the same email to every person in their email list, rather than actually looking at what kind of customer we are.

Sometimes it's even worse than just using our first name on the preamble. I'm sure you've received emails like this before too:

```
Dear $fname,

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Cras semper at urna sit amet imperdiet. Integer rutrum tempor tellus et porta. Mauris elit dui, euismod a ipsum sit amet, ultricies dictum ex. Suspendisse bibendum suscipit leo sed egestas. Suspendisse potenti. Maecenas placerat lectus eget tincidunt posuere. Ut aliquet elit ante. Duis vel volutpat massa, pulvinar condimentum enim. Aliquam erat volutpat. Sed vulputate pretium mauris, eu tempus arcu lacinia sed. Aliquam lobortis massa at ligula aliquet tincidunt. Aliquam ultrices sem a ultricies pulvinar. In facilisis pretium cursus. Vestibulum et malesuada justo. Vivamus erat ex, aliquet nec porttitor eu, sodales consectetur dolor.

Sincerely,
Marketing Person
@ Company
```

Below is a simplistic example of creating that email:

In [ ]:
people = ["Alice", "Bob", "Charlie", "David", "Emma"]

for person in people:
    message = f"Hey {person},\n Check out our web analytics platform, it's Awesome! It's perfect for your needs. Buy it now!\n - Marketer John"
    print(message)

# ChatGPT has entered the chat

If there's something that will help us stand out as a marketer, it's when we send emails that are roughly the same in content but we use ChatGPT to generate the actual content. Below we'll see how we could prompt OpenAI for some "unique" emails to send out to people.

## Setup OpenAI

Below we'll install OpenAI's Python module, enter in our API key, define the list of people (listed above), and then the system prompt.

The system prompt helps prime ChatGPT to respond with content that is helpful for us. Notice that we explicity request for ChatGPT to lookup the top 5 web analytics companies in order to give it a small corpus of content to work from for this example.

In [ ]:
!pip install openai --quiet

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [ ]:
system = 'You are a helpful assistant. My name is Marketer John. \
    You help write the body of an email for a fictitious company called "Awesome Web Analytics". \
    This is a web analytics company that is similar to the top 5 web analytics companies \
    (Perform a web search to determine current top 5 web analytics companies). \
    The goal is to write a custom email to users to get them to be interested in our services. \
    The email should be less than 150 words. Address the user by name. End with my signature.'

## Loop through and request

Now we'll loop through each person and generate the emails by querying ChatGPT, storing the responses in a Python list.

In [ ]:
import openai

def chatgpt_generate_email(prompt, person):
    conversation = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": person},
        {"role": "assistant", "content": ""}
    ]

    # Call the OpenAI Chat API
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo-16k",
        messages = conversation,
        temperature = 0.7,
        max_tokens = 800,
        top_p = 1,
        frequency_penalty = 0,
        presence_penalty = 0
    )

    assistant_reply = response.choices[0].message["content"]
    return assistant_reply

In [ ]:
# Define a list to store the responses
emails = []

# Loop through each person and generate the conversation
for person in people:
    # Generate the email
    reply = chatgpt_generate_email(system, person)
    # Store the response in the dictionary
    emails.append({"person": person, "assistant_reply": reply})

## Print out the list

Now we print out the emails, with names and everything. Note that each of the responses is unique and completely random.

In [ ]:
# Print the responses
for email in emails:
    person = email["person"]
    assistant_reply = email["assistant_reply"]
    print(f"Person: {person}")
    print(f"{assistant_reply}")
    print("_"*100)

# Customizing the content to user behavior

Enhancing our performance can be achieved readily when we have access to customer behavior data within our application. For example, what if a user had registered for our service but had not yet completed the installation of the analytics tracking code on their website?

We have a MongoDB Database populated with some fake user data that looks like this:

```json
{
    '_id': ObjectId('64afb3fda9295d8421e7a19f'),
    'first_name': 'James',
    'last_name': 'Villanueva',
    'company_name': 'Foley-Turner',
    'stage': 'generating a tracking code',
    'created_date': 1987-11-09T12:43:26.000+00:00
}
```

Take particular note of the key `stage`, we will be using that to help identify user behavior and have ChatGPT enhance our emails.

Below we use MongoDB as a source for user behavior, then use that to have ChatGPT write some more useful emails for the users.

## Setup MongoDB Atlas

We’ll use MongoDB Atlas in an M0 Sandbox. We'll configure an **admin** user with **atlasAdmin** privileges under **Database Access**. We’ll temporarily allow access from anywhere (IP Address 0.0.0.0/0) under **Network Access**. We’ll note down the **username**, **password** and **host**.

## Load the data into MongoDB Atlas

Next, we'll load the sample data into MongoDB Atlas. We’ll create a new database called `mktg_email_demo` with a collection called `customers`. We’ll load data into this collection. The data load can be accomplished in several ways, such as using MongoDB Compass or mongoimport. The dataset can be found on [GitHub](https://github.com/singlestore-labs/vector-dbs-ai-apps/tree/main/ch6).

## Load the data from MongoDB Atlas

Below, we'll connect to MongoDB Atlas. We’ll replace `<username>`, `<password>` and `<host>` with the values that we saved earlier from MongoDB Atlas.

In [ ]:
!pip3 install pymongo --quiet

In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb+srv://<username>:<password>@<host>/?retryWrites=true&w=majority")
mongo_db = client["mktg_email_demo"]
collection = mongo_db["customers"]

stages = [
    "getting started",
    "generating a tracking code",
    "adding tracking to your website",
    "real-time analytics",
    "conversion tracking",
    "funnels",
    "user segmentation",
    "custom event tracking",
    "data export",
    "dashboard customization"
]

def find_next_stage(current_stage):
    current_index = stages.index(current_stage)
    if current_index < len(stages) - 1:
        return stages[current_index + 1]
    else:
        return stages[current_index]

## Loop through the users in the collection

Limiting to 3 users, we'll loop through each record in our customers collection in MongoDB Atlas.

In [ ]:
limit = 3
count = 0
emails = []

for record in collection.find():
    if count != limit:
        fname = record["first_name"]
        stage = record["stage"]

        next_stage = find_next_stage(stage)
        system = 'You are a helpful assistant, who works for me, Marketer John at Awesome Web Analytics. \
            You help write the body of an email for a fictitious company called "Awesome Web Analytics". \
            We are a web analytics company that is similar to the top 5 web analytics companies \
            (Perform a web search to determine current top 5 web analytics companies). \
            We have users that are at various stages of the pipeline of using our product and we want to \
            send them helpful emails to get them to use our product more. \
            Please write an email for {} who is on stage {} of the on-boarding process. The next stage is {}. \
            Ensure that the email describes the benefits of moving to the next stage. \
            Limit the email to 1 paragraph. End email with my signature.'.format(fname, stage, next_stage)

        reply = chatgpt_generate_email(system, person)
        emails.append({"fname": fname, "stage": stage, "next_stage": next_stage, "email": reply})

        count += 1

## Print out the list

Now let's take a look at what those emails look like now.

In [ ]:
# Print the responses
for e in emails:
    print(f"First Name: {e['fname']}\n")
    print(f"Stage: {e['stage']}\n")
    print(f"Next Stage: {e['next_stage']}\n")
    print(f"{e['email']}\n")
    print("_"*100)

# What about adding in vectors?

It would be useful to be able to drive users to documentation to ensure that they are able to complete what they're working on. So let's use a Vector Database (Pinecone) to store documentation, then perform a query against it to determine the best place in our documentation to send our users.

## Setup Pinecone

In the SingleStoreDB Notebook environment, add the following to the Firewall to allow communication with the Pinecone environment:

- `controller.us-west1-gcp.pinecone.io`
- `*.pinecone.io`
- `*.*.pinecone.io`

In [ ]:
!pip install pinecone-client --quiet

In [ ]:
import getpass

os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")
api_key = os.getenv("PINECONE_API_KEY")

In [ ]:
import pinecone

pinecone.init(
    api_key = api_key,
    environment = "gcp-starter"
)

pinecone.delete_index("web-app-docs")
pinecone.create_index("web-app-docs", dimension = 1536, metric = "euclidean")
pine_index = pinecone.Index("web-app-docs")

## Create embeddings for our stages

We need embeddings for each of the stages in our pipeline, so we can perform searches in Pinecone for the best docs.

In [ ]:
model_id = "text-embedding-ada-002"

def getEmbeddings(text, model_id):

    try:
        text = text.replace("\n", " ")
        response = openai.Embedding.create(input=text,model=model_id)

        embedding = response["data"][0]["embedding"]
        tokens = response["usage"]["total_tokens"]
        status = "success"
        return embedding, tokens, status
    except Exception as e:
        print(e)
        embedding = ""
        tokens = 0
        status = "failed"
        return embedding, tokens, status

In [ ]:
stages_w_embed = []

for s in stages:
    embedding,tokens,status = getEmbeddings(s, model_id)
    stages_w_embed.append({"stage": s, "embedding": embedding})

In [ ]:
id_counter = 1

for s in stages:
    embedding,tokens,status = getEmbeddings(s, model_id)

    if id_counter <= 1:
        parent = 1
    else:
        parent = id_counter - 1

    pine_index.upsert([
        {"id": str(id_counter),
         "values": embedding,
         "metadata": {"content": s, "parent": parent}}
    ])
    
    id_counter += 1

## Search Pinecone for closest result

Using the embeddings for each stage, generated in the last step, we can search the Pinecone index for the closest matching document in our docs.

In [ ]:
def search_pinecone(embedding):
    response = pine_index.query(
        vector = [embedding],
        top_k = 1,
        include_metadata = True
    )
    metadata = response["matches"][0]["metadata"]
    content = metadata["content"]
    permalink = metadata["parent"]
    return content, permalink

## Loop through the users in the collection

Limiting to 5 users, we'll loop through each record in our customers collection in MongoDB Atlas.

In [ ]:
limit = 5
count = 0
emails = []

for record in collection.find():
    if count != limit:
        fname = record["first_name"]
        stage = record["stage"]

        next_stage = find_next_stage(stage)
        this_stage = next((item for item in stages_w_embed if item["stage"] == stage), None)
        next_stage = next((item for item in stages_w_embed if item["stage"] == next_stage), None)

        cur_content, cur_permalink = search_pinecone(this_stage["embedding"])
        next_content, next_permalink = search_pinecone(next_stage["embedding"])

        cur_link = f"https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{cur_permalink}.md"
        next_link = f"https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{next_permalink}.md"

        system = 'You are a helpful assistant. I am Marketer John at Awesome Web Analytics. \
            We are similar to the current top web analytics companies \
            We have users that are at various stages in using our product and we want to \
            send them helpful emails to get them to use our product more. \
            Write an email for {} who is on stage {} of the on-boarding process. The next stage is {}. \
            Ensure the email describes the benefits of moving to the next stage, \
            then always share this link: https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{}.md . \
            Limit the email to 1 paragraph. \
            End email with my signature "Best Regards, \n Marketer John.'.format(fname, stage, next_stage['stage'], next_permalink)

        reply = chatgpt_generate_email(system, person)
        emails.append({"fname": fname, "stage": stage, "next_stage": next_stage["stage"], "email": reply})
        
        count += 1

## Print out the list

Now, let's print out the emails.

In [ ]:
for e in emails:
    print(f"First Name: {e['fname']}\n")
    print(f"Stage: {e['stage']}\n")
    print(f"Next Stage: {e['next_stage']}\n")
    print(f"{e['email']}\n")
    print("_"*100)

# Lets make this easier with SingleStoreDB

## Ingest data live into SingleStoreDB

Before we go on to the steps of querying our data and simplifying the process down to one database, let's ingest the MongoDB data into SingleStoreDB.

First we'll create a link between the MongoDB Atlas hosts that hold our data and SingleStoreDB. We'll use the **SQL Editor** for these commands.

We'll replace `<username>` and `<password>` with the values that we saved earlier from MongoDB Atlas. We'll also need to replace the values for `<primary>`, `<secondary>` and `<secondary>` with the full address for each from MongoDB Atlas.

```sql
CREATE DATABASE IF NOT EXISTS mktg_email_demo;

USE mktg_email_demo;

CREATE LINK mktg_email_demo.link AS MONGODB
CONFIG '{"mongodb.hosts": "<primary>:27017, <secondary>:27017, <secondary>:27017",
        "collection.include.list": "mktg_email_demo.*",
        "mongodb.ssl.enabled": "true",
        "mongodb.authsource": "admin",
        "mongodb.members.auto.discover": "false"}'
CREDENTIALS '{"mongodb.user": "<username>",
            "mongodb.password": "<password>"}';

CREATE TABLES AS INFER PIPELINE AS LOAD DATA LINK mktg_email_demo.link '*' FORMAT AVRO;
```

## Model `customers` table and connect to SingleStoreDB

In [ ]:
from sqlalchemy import create_engine, Column, String, select
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import func
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.orm import Session

# Define the Customer model
Base = declarative_base()

class Customer(Base):
    __tablename__ = "customers"
    _id = Column(JSONB, primary_key = True)
    _more = Column(JSONB)

# Create an engine and session
conn = create_engine(connection_url)
Session = sessionmaker(bind = conn)
session = Session()

## Create embeddings for our stages

In [ ]:
import pandas as pd

df_list = []
id_counter = 1

for s in stages:
    embedding,tokens,status = getEmbeddings(s, model_id)

    if id_counter <= 1:
        parent = None
    else:
        parent = id_counter - 1
    
    stage_df = pd.DataFrame(
        {"id": [id_counter],
         "content": [s],
         "embedding": [embedding],
         "parent": [parent]}
    )

    df_list.append(stage_df)
    
    id_counter += 1

df = pd.concat(df_list, ignore_index = True)

In [ ]:
%%sql
USE mktg_email_demo;

DROP TABLE IF EXISTS docs_splits;

CREATE TABLE IF NOT EXISTS docs_splits (
    id INT,
    content TEXT,
    embedding BLOB,
    parent INT
);

In [ ]:
import struct

def data_to_binary(data: list[float]):
    format_string = "f" * len(data)
    return struct.pack(format_string, *data)

df["embedding"] = df["embedding"].apply(data_to_binary)

In [ ]:
df.to_sql(
    "docs_splits",
    con = conn,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

## Search SingleStoreDB for closest result

In [ ]:
def search_s2(vector):
    query = """
    SELECT content, parent
    FROM (
            SELECT content, parent,
            EUCLIDEAN_DISTANCE(embedding, JSON_ARRAY_PACK('{}')) AS score
            FROM docs_splits
            ORDER BY score
            LIMIT 1
    ) results
    """.format(str(vector))
    result = session.execute(query)
    return result.fetchone()

## Loop through the users in the `customers` table

Limiting to 5 users, we'll loop through each record in our customers table.

In [ ]:
limit = 5
count = 0
emails = []

# Query the Customer model using SQLAlchemy
customers_query = session.query(Customer._more).limit(limit)

for customer in customers_query:
    if count != limit:
        # Process each customer as desired
        fname = customer._more["first_name"]
        stage = customer._more["stage"]
        next_stage = find_next_stage(stage)

        this_stage = next((item for item in stages_w_embed if item["stage"] == stage), None)
        next_stage = next((item for item in stages_w_embed if item["stage"] == next_stage), None)

        cur_content, cur_permalink = search_s2(this_stage["embedding"])
        next_content, next_permalink = search_s2(next_stage["embedding"])

        cur_link = f"https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{cur_permalink}.md"
        next_link = f"https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{next_permalink}.md"

        system = 'You are a helpful assistant. I am Marketer John at Awesome Web Analytics. \
            We are similar to the current top web analytics companies \
            We have users that are at various stages in using our product and we want to \
            send them helpful emails to get them to use our product more. \
            Write an email for {} who is on stage {} of the on-boarding process. The next stage is {}. \
            Ensure the email describes the benefits of moving to the next stage, \
            then always share this link: https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{}.md . \
            Limit the email to 1 paragraph. \
            End email with my signature "Best Regards, \n Marketer John.'.format(fname, stage, next_stage['stage'], next_permalink)
        
        reply = chatgpt_generate_email(system, person)
        emails.append({"fname": fname, "stage": stage, "next_stage": next_stage["stage"], "email": reply})

        count += 1

session.commit()
session.close()

## Print out the list

Now, let's print out the emails.

In [ ]:
for e in emails:
    print(f"First Name: {e['fname']}\n")
    print(f"Stage: {e['stage']}\n")
    print(f"Next Stage: {e['next_stage']}\n")
    print(f"{e['email']}\n")
    print("_"*100)

# Conclusion

By combining traditional marketing techniques with ChatGPT and powerful database functions, we can automate the writing of effective emails that are contextually important to our users, without the need to spend hours diving into the analytics ourselves.

We can see in this demonstration that we can build a solution using multiple database systems (MongoDB and Pinecone), or we can do this with SingleStoreDB, a powerful database system that combines traditional Relational and NoSQL options with Vectors, enabling us to write our queries in familiar SQL syntax.